# Day 4 – Integrating Results into Programs

## Install required libraries

In [ ]:
!pip install qiskit qiskit-ibm-runtime networkx

In [ ]:
# Imports
from qiskit import Aer, QuantumCircuit
from qiskit.visualization import plot_histogram
from qiskit_ibm_runtime import QiskitRuntimeService, Estimator, Session
from qiskit.circuit.library import TwoLocal, QAOAAnsatz
from qiskit.algorithms.optimizers import COBYLA
from qiskit.primitives import Sampler
from qiskit.quantum_info import Statevector
import networkx as nx
import matplotlib.pyplot as plt
import json


## Step 1: Build the MaxCut Problem

In [ ]:
# Create a small graph
G = nx.Graph()
G.add_edges_from([(0, 1), (1, 2), (2, 3), (3, 0)])
nx.draw(G, with_labels=True)


In [ ]:
# Define MaxCut cost Hamiltonian
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.applications import Maxcut

maxcut = Maxcut(G)
qp = maxcut.to_quadratic_program()

print(qp.export_as_lp_string())


## Step 2: Convert to QUBO and Build QAOA Circuit

In [ ]:
from qiskit_optimization.algorithms import QAOA
from qiskit_optimization.converters import QuadraticProgramToQubo

qubo = QuadraticProgramToQubo().convert(qp)

sampler = Sampler()
optimizer = COBYLA(maxiter=50)

qaoa = QAOA(sampler=sampler, optimizer=optimizer, reps=2)
result = qaoa.compute_minimum_eigenvalue(qubo.to_ising()[0])

print("QAOA result:", result)


## Step 3: Run on IBM Quantum Backend

In [ ]:
# Connect to IBM Quantum
# Replace with your own API token
QiskitRuntimeService.save_account(channel="ibm_quantum", token="YOUR_API_TOKEN", overwrite=True)
service = QiskitRuntimeService()

# Select backend
backend = service.backend("ibm_osaka")  # replace with your available backend
session = Session(service=service, backend=backend)

estimator = Estimator(session=session)
qaoa_runtime = QAOA(estimator=estimator, optimizer=optimizer, reps=2)
result_real = qaoa_runtime.compute_minimum_eigenvalue(qubo.to_ising()[0])

session.close()

# Export results as JSON
results_json = json.dumps({"optimal_value": result_real.eigenvalue.real})
print("Exported Results:", results_json)


## Step 4: Use Results in Python Code

In [ ]:
# Example: Using QAOA output to determine the cut set
bitstring = maxcut.sample_most_likely(result_real.eigenstate)
cut_value = maxcut.max_cut_value(bitstring, G)

print("Best bitstring:", bitstring)
print("Cut value:", cut_value)

# Use in your Python logic
if cut_value > 2:
    print("High cut found. Graph strongly bipartite.")
else:
    print("Low cut found. Graph weakly connected.")


## Step 5: Student-Designed Circuits Integration

In [ ]:
# Example of a student-designed circuit
student_qc = QuantumCircuit(2)
student_qc.h(0)
student_qc.cx(0, 1)
student_qc.measure_all()

# Run on simulator
sim = Aer.get_backend("qasm_simulator")
sim_result = sim.run(student_qc, shots=1024).result().get_counts()
print("Simulator counts:", sim_result)

# Integrate with Python logic
if sim_result.get("11", 0) > 400:
    print("Circuit produced strong entanglement.")
else:
    print("Weak entanglement detected.")
